**Лабораторная работа №4. Выделение контуров на изображении**

In [ ]:
from PIL import Image, ImageDraw
import numpy as np
import math
import random
from scipy import signal
import os

Выполняется одно задание на выбор. В качестве входных данных берётся цветное
или полутоновое изображение. Цветное изображение сводится к полутоновому, вариант 9.
Демонстрируется:

1) Исходное цветное изображение.

2) Полутоновое изображение.

3) Три градиентные матрицы Gx, Gy, G, нормализованные так, что значения
яркости лежат от 0 до 255.

4) Бинаризованная градиентная матрица G, где порог подбирается опытным
путём.


In [ ]:
def monochrome(input_array):
  height, width = input_array.shape[:2]
  output_array = np.zeros((height, width), dtype=input_array.dtype)

  for y in range(height):
      for x in range(width):
          output_array[y][x] = np.mean(input_array[y][x])

  #return pim.fromarray(output_array, mode='L')
  new_image = Image.fromarray(output_array)
  new_image.save('new_image_monochrome.png')


In [ ]:
def get_pixel(pixel_grad, T) :
  if pixel_grad > T :
    return 255
  else :
    return 0

def pruitt_operator(input_array, T):
  height, width = input_array.shape[:2]
  input_array = np.array(input_array)

  # Оператор Прюитт 5x5
  kernel_x = np.array([[-1, -1, -1, -1, -1],
                      [0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0],
                      [1, 1, 1, 1, 1]])

  kernel_y = np.array([[-1, 0, 0, 0, 1],
                      [-1, 0, 0, 0, 1],
                      [-1, 0, 0, 0, 1],
                      [-1, 0, 0, 0, 1],
                      [-1, 0, 0, 0, 1]])

  # Рассчет Gx и Gy
  gradient_x = signal.convolve2d(input_array, kernel_x, boundary='symm', mode='same')
  gradient_y = signal.convolve2d(input_array, kernel_y, boundary='symm', mode='same')

  # Поиск градиента
  gradient = np.abs(gradient_x) + np.abs(gradient_y)

  # Нормализация градиента Gx
  normalized_gradient_x = (gradient_x / np.max(gradient_x)) * 255
  normalized_gradient_x = normalized_gradient_x.astype(np.uint8)
  new_image = Image.fromarray(normalized_gradient_x)
  new_image.save('new_image_normalized_gradient_x.png')

  # Нормализация градиента Gy
  normalized_gradient_y = (gradient_y / np.max(gradient_y)) * 255
  normalized_gradient_y = normalized_gradient_y.astype(np.uint8)
  new_image = Image.fromarray(normalized_gradient_y)
  new_image.save('new_image_normalized_gradient_y.png')

  # Нормализация градиента G
  normalized_gradient = (gradient / np.max(gradient)) * 255
  normalized_gradient = normalized_gradient.astype(np.uint8)
  new_image = Image.fromarray(normalized_gradient)
  new_image.save('new_image_normalized_gradien.png')

  # Применение оператора Пьюитта к изображению
  output_array = np.zeros_like(input_array)
  normalized_gradient = (gradient / np.max(gradient)) * 255
  for x in range(input_array.shape[0]) :
    for y in range(input_array.shape[1]) :
      output_array[x, y] = get_pixel(normalized_gradient[x, y], T)

  output_image = Image.fromarray(output_array.astype(np.uint8), 'L')
  output_image.save('new_image_pruitt_operator.png')

In [ ]:
def main():
  input_image = Image.open('japan.png')
  input_array = np.array(input_image)

  monochrome(input_array)

  input_image = Image.open('new_image_monochrome.png')
  input_array = np.array(input_image)

  T = 20
  pruitt_operator(input_array, T)

if __name__ == '__main__':
    main()